# Partie 2.1 - Nettoyage des donnees meteorologiques avec Pandas

Ce notebook realise le nettoyage complet des donnees meteorologiques brutes issues du fichier `meteo_raw.csv`.  
Les etapes couvrent : exploration, standardisation des formats, correction des valeurs aberrantes,  
traitement des valeurs manquantes, enrichissement temporel et sauvegarde du fichier nettoye.

In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import warnings
import os

warnings.filterwarnings('ignore')

# Affichage elargi pour ne pas tronquer les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 60)

print("Pandas version :", pd.__version__)
print("NumPy version  :", np.__version__)

Pandas version : 2.3.3
NumPy version  : 2.2.6


## 1. Chargement et exploration initiale

In [2]:
# Chargement du fichier brut
chemin_fichier = os.path.join("..", "data", "meteo_raw.csv")
df = pd.read_csv(chemin_fichier, dtype=str)  # Lecture en str pour controler les conversions

print("="*70)
print("EXPLORATION INITIALE DU JEU DE DONNEES")
print("="*70)

# Dimensions du DataFrame
print(f"\nDimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")

# Informations generales
print("\n--- Informations generales ---")
print(df.info())

# Apercu des premieres lignes
print("\n--- Apercu des 10 premieres lignes ---")
display(df.head(10))

# Statistiques descriptives (sur colonnes str)
print("\n--- Statistiques descriptives ---")
display(df.describe(include='all'))

# Nombre de valeurs nulles par colonne
print("\n--- Valeurs nulles par colonne ---")
nulls = df.isnull().sum()
for col in df.columns:
    pct = nulls[col] / len(df) * 100
    print(f"  {col:30s} : {nulls[col]:>6d} nulls ({pct:.2f}%)")

# Liste des communes presentes
print(f"\nNombre de communes : {df['commune'].nunique()}")
print("Communes :", sorted(df['commune'].dropna().unique()))

EXPLORATION INITIALE DU JEU DE DONNEES

Dimensions : 252612 lignes x 7 colonnes

--- Informations generales ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252612 entries, 0 to 252611
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   commune                  252612 non-null  object
 1   timestamp                252612 non-null  object
 2   temperature_c            251383 non-null  object
 3   humidite_pct             252612 non-null  object
 4   rayonnement_solaire_wm2  252612 non-null  object
 5   vitesse_vent_kmh         252612 non-null  object
 6   precipitation_mm         252612 non-null  object
dtypes: object(7)
memory usage: 13.5+ MB
None

--- Apercu des 10 premieres lignes ---


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm
0,Saint-Etienne,09/15/2024 15:00:00,17.1,143.3,244.9,14.3,0.0
1,Bordeaux,21/07/2023 15:00,19.6,50.6,414.9,3.2,0.0
2,Montpellier,2023-09-18 20:00:00,18.3,65.7,218.4,13.6,0.0
3,Le Havre,01/03/2024 22:00:00,3.7,94.9,6.8,18.6,11.6
4,Lille,29/10/2024 20:00,14.0,42.9,781.8,4.0,0.0
5,Bordeaux,22/12/2023 13:00,4.4,36.9,796.4,6.1,0.0
6,Marseille,09/15/2023 21:00:00,22.5,86.8,5.8,32.6,0.0
7,Toulouse,30/05/2023 00:00,8.3,66.3,26.4,31.4,7.2
8,Bordeaux,2024-10-05T09:00:00,11.5,69.8,71.4,34.4,0.0
9,Toulon,2024-09-28T21:00:00,19.2,79.0,13.1,31.8,0.0



--- Statistiques descriptives ---


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm
count,252612,252612,251383,252612,252612,252612,252612
unique,15,69066,810,1094,8685,401,151
top,Saint-Etienne,24/02/2024 16:00,9.3,69.9,10.3,5.5,0.0
freq,16876,12,1216,449,254,702,189528



--- Valeurs nulles par colonne ---
  commune                        :      0 nulls (0.00%)
  timestamp                      :      0 nulls (0.00%)
  temperature_c                  :   1229 nulls (0.49%)
  humidite_pct                   :      0 nulls (0.00%)
  rayonnement_solaire_wm2        :      0 nulls (0.00%)
  vitesse_vent_kmh               :      0 nulls (0.00%)
  precipitation_mm               :      0 nulls (0.00%)

Nombre de communes : 15
Communes : ['Bordeaux', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Nantes', 'Nice', 'Paris', 'Reims', 'Rennes', 'Saint-Etienne', 'Strasbourg', 'Toulon', 'Toulouse']


## 2. Rapport de completude avant nettoyage

In [3]:
def rapport_completude(dataframe, titre="Rapport de completude"):
    """
    Genere un rapport de completude pour chaque colonne du DataFrame.
    Retourne un DataFrame avec le nombre de valeurs non-nulles, nulles
    et le pourcentage de completude.
    """
    n_total = len(dataframe)
    rapport = []
    
    for col in dataframe.columns:
        n_non_null = dataframe[col].notna().sum()
        n_null = dataframe[col].isna().sum()
        pct_complet = (n_non_null / n_total) * 100 if n_total > 0 else 0.0
        rapport.append({
            'colonne': col,
            'non_null': n_non_null,
            'null': n_null,
            'pct_completude': round(pct_complet, 2)
        })
    
    df_rapport = pd.DataFrame(rapport)
    df_rapport = df_rapport.set_index('colonne')
    
    print("="*70)
    print(titre.upper())
    print("="*70)
    print(f"Nombre total de lignes : {n_total}")
    display(df_rapport)
    
    return df_rapport


# Rapport avant nettoyage
rapport_avant = rapport_completude(df, "Rapport de completude AVANT nettoyage")

RAPPORT DE COMPLETUDE AVANT NETTOYAGE
Nombre total de lignes : 252612


,non_null,null,pct_completude
colonne,,,
commune,252612,0,100.00
timestamp,252612,0,100.00
temperature_c,251383,1229,99.51
humidite_pct,252612,0,100.00
rayonnement_solaire_wm2,252612,0,100.00
vitesse_vent_kmh,252612,0,100.00
precipitation_mm,252612,0,100.00


## 3. Standardisation des formats de dates

In [4]:
def parser_timestamp_multi_format(valeur):
    """
    Tente de convertir une valeur timestamp en datetime en essayant
    plusieurs formats courants :
      - ISO standard  : 2023-01-15 08:00:00
      - ISO avec T     : 2023-01-15T08:00:00
      - Format francais: 15/01/2023 08:00:00
      - Format americain: 01/15/2023 08:00:00
    Retourne NaT si aucun format ne correspond.
    """
    if pd.isna(valeur) or str(valeur).strip() == '':
        return pd.NaT
    
    valeur_str = str(valeur).strip()
    
    # Liste des formats a essayer dans l'ordre de priorite
    formats = [
        '%Y-%m-%d %H:%M:%S',    # ISO standard
        '%Y-%m-%dT%H:%M:%S',    # ISO avec separateur T
        '%d/%m/%Y %H:%M:%S',    # Format francais (jour/mois/annee)
        '%m/%d/%Y %H:%M:%S',    # Format americain (mois/jour/annee)
        '%Y-%m-%d',             # ISO date seule
        '%d/%m/%Y',             # FR date seule
        '%m/%d/%Y',             # US date seule
    ]
    
    for fmt in formats:
        try:
            dt = pd.to_datetime(valeur_str, format=fmt)
            return dt
        except (ValueError, TypeError):
            continue
    
    # Dernier recours : laisser pandas deviner
    try:
        return pd.to_datetime(valeur_str, dayfirst=True)
    except (ValueError, TypeError):
        return pd.NaT


# Nombre de timestamps avant conversion
n_total = len(df)
n_ts_null_avant = df['timestamp'].isna().sum()
print(f"Timestamps avant conversion : {n_total} lignes, {n_ts_null_avant} nulls")

# Apercu des differents formats presents
print("\nExemples de formats detectes :")
echantillon = df['timestamp'].dropna().sample(min(20, len(df)), random_state=42)
for val in echantillon.values:
    print(f"  {val}")

# Application de la fonction de parsing multi-format
print("\nConversion en cours...")
df['timestamp'] = df['timestamp'].apply(parser_timestamp_multi_format)

# Statistiques de conversion
n_ts_null_apres = df['timestamp'].isna().sum()
n_convertis = n_total - n_ts_null_apres
taux_succes = (n_convertis / n_total) * 100

print(f"\n--- Resultat de la standardisation des dates ---")
print(f"  Timestamps convertis avec succes : {n_convertis} / {n_total} ({taux_succes:.2f}%)")
print(f"  Timestamps non convertis (NaT)   : {n_ts_null_apres}")
print(f"  Plage temporelle : {df['timestamp'].min()} -> {df['timestamp'].max()}")

# Suppression des lignes sans timestamp valide
n_avant = len(df)
df = df.dropna(subset=['timestamp'])
n_supprimes = n_avant - len(df)
print(f"\n  Lignes supprimees (timestamp invalide) : {n_supprimes}")
print(f"  Lignes restantes : {len(df)}")

Timestamps avant conversion : 252612 lignes, 0 nulls

Exemples de formats detectes :
  12/02/2023 17:00:00
  06/12/2024 01:00
  07/11/2023 23:00:00
  11/24/2023 21:00:00
  01/30/2023 09:00:00
  09/14/2023 12:00:00
  2024-01-07T21:00:00
  05/09/2023 15:00
  06/02/2023 08:00:00
  2024-05-10T05:00:00
  2024-10-27T09:00:00
  04/21/2024 12:00:00
  14/09/2023 03:00
  2023-01-23T10:00:00
  08/21/2024 11:00:00
  12/10/2024 20:00
  2023-11-25T10:00:00
  19/02/2024 02:00
  10/15/2024 23:00:00
  2024-09-15 17:00:00

Conversion en cours...

--- Resultat de la standardisation des dates ---
  Timestamps convertis avec succes : 252612 / 252612 (100.00%)
  Timestamps non convertis (NaT)   : 0
  Plage temporelle : 2023-01-01 00:00:00 -> 2024-12-31 23:00:00

  Lignes supprimees (timestamp invalide) : 0
  Lignes restantes : 252612


## 4. Conversion des colonnes numeriques

In [5]:
def convertir_colonne_numerique(serie, nom_colonne):
    """
    Convertit une colonne texte en float :
      1. Remplace les virgules decimales par des points
      2. Remplace les valeurs textuelles invalides ('', 'NA', 'null', 'None') par NaN
      3. Convertit en float64
    Retourne la serie convertie et un dictionnaire de statistiques.
    """
    stats = {
        'colonne': nom_colonne,
        'total': len(serie),
        'null_avant': serie.isna().sum()
    }
    
    # Travailler sur une copie en string
    s = serie.copy().astype(str)
    
    # Compter les virgules decimales avant remplacement
    masque_virgule = s.str.contains(',', na=False) & ~s.str.contains('/', na=False)
    stats['virgules_remplacees'] = masque_virgule.sum()
    
    # Etape 1 : Remplacer les virgules decimales par des points
    s = s.str.replace(',', '.', regex=False)
    
    # Etape 2 : Identifier et remplacer les valeurs textuelles invalides
    valeurs_invalides = ['', 'na', 'null', 'none', 'nan', 'n/a', '-', '--']
    masque_texte = s.str.strip().str.lower().isin(valeurs_invalides)
    stats['textes_remplaces'] = masque_texte.sum()
    s[masque_texte] = np.nan
    
    # Remettre les vrais NaN d'origine
    s[serie.isna()] = np.nan
    
    # Etape 3 : Conversion en float
    serie_convertie = pd.to_numeric(s, errors='coerce')
    
    stats['null_apres'] = serie_convertie.isna().sum()
    stats['convertis_ok'] = stats['total'] - stats['null_apres']
    stats['pct_succes'] = round((stats['convertis_ok'] / stats['total']) * 100, 2)
    
    return serie_convertie, stats


# Colonnes numeriques a convertir
colonnes_numeriques = [
    'temperature_c',
    'humidite_pct',
    'rayonnement_solaire_wm2',
    'vitesse_vent_kmh',
    'precipitation_mm'
]

print("="*70)
print("CONVERSION DES COLONNES NUMERIQUES")
print("="*70)

stats_conversion = []

for col in colonnes_numeriques:
    if col in df.columns:
        df[col], stats = convertir_colonne_numerique(df[col], col)
        stats_conversion.append(stats)
        print(f"\n  {col}:")
        print(f"    Virgules decimales corrigees : {stats['virgules_remplacees']}")
        print(f"    Valeurs textuelles remplacees: {stats['textes_remplaces']}")
        print(f"    Taux de conversion reussi    : {stats['pct_succes']}%")
    else:
        print(f"  [AVERTISSEMENT] Colonne '{col}' absente du DataFrame.")

# Tableau recapitulatif
print("\n--- Tableau recapitulatif des conversions ---")
df_stats = pd.DataFrame(stats_conversion).set_index('colonne')
display(df_stats)

# Verification des types apres conversion
print("\nTypes des colonnes apres conversion :")
print(df[colonnes_numeriques].dtypes)

CONVERSION DES COLONNES NUMERIQUES

  temperature_c:
    Virgules decimales corrigees : 19851
    Valeurs textuelles remplacees: 1229
    Taux de conversion reussi    : 99.51%

  humidite_pct:
    Virgules decimales corrigees : 0
    Valeurs textuelles remplacees: 0
    Taux de conversion reussi    : 100.0%

  rayonnement_solaire_wm2:
    Virgules decimales corrigees : 0
    Valeurs textuelles remplacees: 0
    Taux de conversion reussi    : 100.0%

  vitesse_vent_kmh:
    Virgules decimales corrigees : 0
    Valeurs textuelles remplacees: 0
    Taux de conversion reussi    : 100.0%

  precipitation_mm:
    Virgules decimales corrigees : 0
    Valeurs textuelles remplacees: 0
    Taux de conversion reussi    : 100.0%

--- Tableau recapitulatif des conversions ---


,total,null_avant,virgules_remplacees,textes_remplaces,null_apres,convertis_ok,pct_succes
colonne,,,,,,,
temperature_c,252612,1229,19851,1229,1229,251383,99.51
humidite_pct,252612,0,0,0,0,252612,100.00
rayonnement_solaire_wm2,252612,0,0,0,0,252612,100.00
vitesse_vent_kmh,252612,0,0,0,0,252612,100.00
precipitation_mm,252612,0,0,0,0,252612,100.00



Types des colonnes apres conversion :
temperature_c              float64
humidite_pct               float64
rayonnement_solaire_wm2    float64
vitesse_vent_kmh           float64
precipitation_mm           float64
dtype: object


## 5. Correction des valeurs aberrantes

In [6]:
print("="*70)
print("CORRECTION DES VALEURS ABERRANTES")
print("="*70)

corrections = {}

# --- Temperature : valeurs hors [-40, 50] -> NaN (sera interpole ensuite) ---
masque_temp = (df['temperature_c'] < -40) | (df['temperature_c'] > 50)
n_temp_aberrantes = masque_temp.sum()
df.loc[masque_temp, 'temperature_c'] = np.nan
corrections['temperature_c (hors [-40, 50] -> NaN)'] = n_temp_aberrantes
print(f"\n  Temperature hors [-40, 50] remplacees par NaN : {n_temp_aberrantes}")

# --- Humidite : valeurs hors [0, 100] -> clipping a [0, 100] ---
masque_hum_basse = df['humidite_pct'] < 0
masque_hum_haute = df['humidite_pct'] > 100
n_hum_corrigees = masque_hum_basse.sum() + masque_hum_haute.sum()
df['humidite_pct'] = df['humidite_pct'].clip(lower=0, upper=100)
corrections['humidite_pct (clip [0, 100])'] = n_hum_corrigees
print(f"  Humidite clippee a [0, 100]                   : {n_hum_corrigees}")
print(f"    - Valeurs < 0   : {masque_hum_basse.sum()}")
print(f"    - Valeurs > 100 : {masque_hum_haute.sum()}")

# --- Rayonnement solaire negatif -> 0 ---
masque_ray = df['rayonnement_solaire_wm2'] < 0
n_ray_corrigees = masque_ray.sum()
df.loc[masque_ray, 'rayonnement_solaire_wm2'] = 0
corrections['rayonnement_solaire_wm2 (negatif -> 0)'] = n_ray_corrigees
print(f"  Rayonnement solaire negatif mis a 0            : {n_ray_corrigees}")

# --- Vitesse du vent negative -> 0 (par coherence physique) ---
masque_vent = df['vitesse_vent_kmh'] < 0
n_vent_corrigees = masque_vent.sum()
df.loc[masque_vent, 'vitesse_vent_kmh'] = 0
corrections['vitesse_vent_kmh (negatif -> 0)'] = n_vent_corrigees
print(f"  Vitesse du vent negative mise a 0              : {n_vent_corrigees}")

# --- Precipitation negative -> 0 ---
masque_precip = df['precipitation_mm'] < 0
n_precip_corrigees = masque_precip.sum()
df.loc[masque_precip, 'precipitation_mm'] = 0
corrections['precipitation_mm (negatif -> 0)'] = n_precip_corrigees
print(f"  Precipitation negative mise a 0                : {n_precip_corrigees}")

# Tableau recapitulatif
print("\n--- Recapitulatif des corrections ---")
df_corrections = pd.DataFrame(
    list(corrections.items()),
    columns=['regle_correction', 'nombre_valeurs_corrigees']
)
display(df_corrections)
print(f"\nTotal de corrections appliquees : {sum(corrections.values())}")

CORRECTION DES VALEURS ABERRANTES

  Temperature hors [-40, 50] remplacees par NaN : 1977
  Humidite clippee a [0, 100]                   : 1801
    - Valeurs < 0   : 0
    - Valeurs > 100 : 1801
  Rayonnement solaire negatif mis a 0            : 1275
  Vitesse du vent negative mise a 0              : 0
  Precipitation negative mise a 0                : 0

--- Recapitulatif des corrections ---


,regle_correction,nombre_valeurs_corrigees
0,"temperature_c (hors [-40, 50] -> NaN)",1977
1,"humidite_pct (clip [0, 100])",1801
2,rayonnement_solaire_wm2 (negatif -> 0),1275
3,vitesse_vent_kmh (negatif -> 0),0
4,precipitation_mm (negatif -> 0),0



Total de corrections appliquees : 5053


## 6. Traitement des valeurs manquantes

In [7]:
print("="*70)
print("TRAITEMENT DES VALEURS MANQUANTES")
print("="*70)

# Etat des NaN avant traitement
print("\nValeurs manquantes AVANT interpolation :")
nan_avant = df[colonnes_numeriques].isna().sum()
for col in colonnes_numeriques:
    print(f"  {col:30s} : {nan_avant[col]:>6d} NaN")

# Tri par commune et timestamp pour assurer la coherence temporelle
df = df.sort_values(by=['commune', 'timestamp']).reset_index(drop=True)
print("\nDonnees triees par commune et timestamp.")

# --- Interpolation lineaire par commune pour temperature et humidite ---
colonnes_interpolation = ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh']

print("\nInterpolation lineaire par commune pour : temperature, humidite, rayonnement, vent...")
for col in colonnes_interpolation:
    df[col] = df.groupby('commune')[col].transform(
        lambda x: x.interpolate(method='linear', limit_direction='both')
    )

# --- Forward fill pour precipitation (pas de pluie = derniere valeur connue) ---
print("Forward fill par commune pour : precipitation...")
df['precipitation_mm'] = df.groupby('commune')['precipitation_mm'].transform(
    lambda x: x.fillna(method='ffill').fillna(method='bfill')
)

# Etat des NaN apres traitement
print("\nValeurs manquantes APRES interpolation :")
nan_apres = df[colonnes_numeriques].isna().sum()
for col in colonnes_numeriques:
    reduction = nan_avant[col] - nan_apres[col]
    print(f"  {col:30s} : {nan_apres[col]:>6d} NaN (reduction : {reduction})")

# Suppression des lignes restantes avec NaN dans les colonnes critiques
n_avant_drop = len(df)
df = df.dropna(subset=colonnes_numeriques)
n_supprimes = n_avant_drop - len(df)
print(f"\nLignes supprimees (NaN residuels) : {n_supprimes}")
print(f"Lignes restantes : {len(df)}")

TRAITEMENT DES VALEURS MANQUANTES

Valeurs manquantes AVANT interpolation :
  temperature_c                  :   3206 NaN
  humidite_pct                   :      0 NaN
  rayonnement_solaire_wm2        :      0 NaN
  vitesse_vent_kmh               :      0 NaN
  precipitation_mm               :      0 NaN

Donnees triees par commune et timestamp.

Interpolation lineaire par commune pour : temperature, humidite, rayonnement, vent...
Forward fill par commune pour : precipitation...

Valeurs manquantes APRES interpolation :
  temperature_c                  :      0 NaN (reduction : 3206)
  humidite_pct                   :      0 NaN (reduction : 0)
  rayonnement_solaire_wm2        :      0 NaN (reduction : 0)
  vitesse_vent_kmh               :      0 NaN (reduction : 0)
  precipitation_mm               :      0 NaN (reduction : 0)

Lignes supprimees (NaN residuels) : 0
Lignes restantes : 252612


## 7. Ajout de colonnes temporelles

In [8]:
print("="*70)
print("AJOUT DE COLONNES TEMPORELLES")
print("="*70)


def determiner_saison(mois):
    """
    Determine la saison meteorologique a partir du numero de mois.
    Hiver     : decembre (12), janvier (1), fevrier (2)
    Printemps : mars (3), avril (4), mai (5)
    Ete       : juin (6), juillet (7), aout (8)
    Automne   : septembre (9), octobre (10), novembre (11)
    """
    if mois in [12, 1, 2]:
        return 'hiver'
    elif mois in [3, 4, 5]:
        return 'printemps'
    elif mois in [6, 7, 8]:
        return 'ete'
    else:
        return 'automne'


# Mapping des jours de la semaine en francais
jours_fr = {
    0: 'Lundi',
    1: 'Mardi',
    2: 'Mercredi',
    3: 'Jeudi',
    4: 'Vendredi',
    5: 'Samedi',
    6: 'Dimanche'
}

# Extraction des composantes temporelles
df['jour'] = df['timestamp'].dt.day
df['mois'] = df['timestamp'].dt.month
df['heure'] = df['timestamp'].dt.hour
df['saison'] = df['mois'].apply(determiner_saison)
df['jour_semaine'] = df['timestamp'].dt.dayofweek.map(jours_fr)

print("\nColonnes temporelles ajoutees : jour, mois, heure, saison, jour_semaine")

# Verification sur un echantillon
print("\n--- Echantillon avec colonnes temporelles ---")
colonnes_affichage = ['commune', 'timestamp', 'jour', 'mois', 'saison', 'jour_semaine', 'heure', 'temperature_c']
display(df[colonnes_affichage].sample(10, random_state=42))

# Distribution par saison
print("\n--- Repartition par saison ---")
print(df['saison'].value_counts().sort_index())

# Distribution par jour de la semaine
print("\n--- Repartition par jour de la semaine ---")
ordre_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
print(df['jour_semaine'].value_counts().reindex(ordre_jours))

AJOUT DE COLONNES TEMPORELLES

Colonnes temporelles ajoutees : jour, mois, heure, saison, jour_semaine

--- Echantillon avec colonnes temporelles ---


,commune,timestamp,jour,mois,saison,jour_semaine,heure,temperature_c
168489,Rennes,2023-01-05 01:00:00,5,1,hiver,Jeudi,1,8.1
181045,Rennes,2024-07-04 07:00:00,4,7,ete,Jeudi,7,24.2
54509,Lyon,2023-06-22 21:00:00,22,6,ete,Jeudi,21,31.7
236490,Toulouse,2023-02-01 02:00:00,1,2,hiver,Mercredi,2,-0.5
224455,Toulon,2023-08-29 12:00:00,29,8,ete,Mardi,12,31.9
169902,Rennes,2023-03-08 15:00:00,8,3,printemps,Mercredi,15,20.6
51112,Lyon,2023-01-26 21:00:00,26,1,hiver,Jeudi,21,11.8
54750,Lyon,2023-07-03 05:00:00,3,7,ete,Lundi,5,31.7
124093,Nice,2023-09-27 03:00:00,27,9,automne,Mercredi,3,13.6
93581,Montpellier,2024-02-12 21:00:00,12,2,hiver,Lundi,21,7.4



--- Repartition par saison ---
saison
automne      62914
ete          63468
hiver        62550
printemps    63680
Name: count, dtype: int64

--- Repartition par jour de la semaine ---
jour_semaine
Lundi       36286
Mardi       36280
Mercredi    36000
Jeudi       35901
Vendredi    35885
Samedi      35935
Dimanche    36325
Name: count, dtype: int64


## 8. Rapport de completude apres nettoyage

In [9]:
# Rapport apres nettoyage
rapport_apres = rapport_completude(df, "Rapport de completude APRES nettoyage")

# --- Comparaison cote a cote ---
print("\n" + "="*70)
print("COMPARAISON AVANT / APRES NETTOYAGE")
print("="*70)

# Construire le tableau comparatif sur les colonnes communes
colonnes_communes = rapport_avant.index.intersection(rapport_apres.index)

comparaison = pd.DataFrame({
    'pct_avant': rapport_avant.loc[colonnes_communes, 'pct_completude'],
    'pct_apres': rapport_apres.loc[colonnes_communes, 'pct_completude'],
})
comparaison['amelioration_pts'] = comparaison['pct_apres'] - comparaison['pct_avant']

display(comparaison)

print(f"\nNombre de lignes avant nettoyage : {rapport_avant['non_null'].iloc[0] + rapport_avant['null'].iloc[0]}")
print(f"Nombre de lignes apres nettoyage : {len(df)}")
print(f"Nombre de colonnes apres nettoyage : {len(df.columns)}")
print(f"\nNouvelles colonnes ajoutees : jour, mois, heure, saison, jour_semaine")

RAPPORT DE COMPLETUDE APRES NETTOYAGE
Nombre total de lignes : 252612


,non_null,null,pct_completude
colonne,,,
commune,252612,0,100.0
timestamp,252612,0,100.0
temperature_c,252612,0,100.0
humidite_pct,252612,0,100.0
rayonnement_solaire_wm2,252612,0,100.0
vitesse_vent_kmh,252612,0,100.0
precipitation_mm,252612,0,100.0
jour,252612,0,100.0
mois,252612,0,100.0



COMPARAISON AVANT / APRES NETTOYAGE


,pct_avant,pct_apres,amelioration_pts
colonne,,,
commune,100.00,100.0,0.00
timestamp,100.00,100.0,0.00
temperature_c,99.51,100.0,0.49
humidite_pct,100.00,100.0,0.00
rayonnement_solaire_wm2,100.00,100.0,0.00
vitesse_vent_kmh,100.00,100.0,0.00
precipitation_mm,100.00,100.0,0.00



Nombre de lignes avant nettoyage : 252612
Nombre de lignes apres nettoyage : 252612
Nombre de colonnes apres nettoyage : 12

Nouvelles colonnes ajoutees : jour, mois, heure, saison, jour_semaine


## 9. Sauvegarde

In [10]:
# Chemin de sortie
chemin_sortie = os.path.join("..", "output", "meteo_clean.csv")

# Creation du repertoire de sortie si necessaire
os.makedirs(os.path.dirname(chemin_sortie), exist_ok=True)

# Sauvegarde du DataFrame nettoye
df.to_csv(chemin_sortie, index=False, encoding='utf-8')

# Verification
taille_fichier = os.path.getsize(chemin_sortie)
taille_mo = taille_fichier / (1024 * 1024)

print("="*70)
print("SAUVEGARDE DU FICHIER NETTOYE")
print("="*70)
print(f"\n  Chemin            : {os.path.abspath(chemin_sortie)}")
print(f"  Dimensions finales: {df.shape[0]} lignes x {df.shape[1]} colonnes")
print(f"  Taille du fichier : {taille_mo:.2f} Mo ({taille_fichier:,} octets)")
print(f"\n  Colonnes sauvegardees :")
for i, col in enumerate(df.columns, 1):
    print(f"    {i:2d}. {col} ({df[col].dtype})")

print("\nSauvegarde terminee avec succes.")

SAUVEGARDE DU FICHIER NETTOYE

  Chemin            : /Users/ihababadi/Downloads/ECF_ENERGIE_BATIMENTS (1)/ecf_energie/output/meteo_clean.csv
  Dimensions finales: 252612 lignes x 12 colonnes
  Taille du fichier : 17.89 Mo (18,753,910 octets)

  Colonnes sauvegardees :
     1. commune (object)
     2. timestamp (datetime64[ns])
     3. temperature_c (float64)
     4. humidite_pct (float64)
     5. rayonnement_solaire_wm2 (float64)
     6. vitesse_vent_kmh (float64)
     7. precipitation_mm (float64)
     8. jour (int32)
     9. mois (int32)
    10. heure (int32)
    11. saison (object)
    12. jour_semaine (object)

Sauvegarde terminee avec succes.


## Conclusion

Ce notebook a realise le nettoyage complet des donnees meteorologiques brutes. Voici le resume des operations effectuees :

1. **Chargement** : Lecture du fichier `meteo_raw.csv` contenant les donnees horaires de 15 communes sur 2023-2024.

2. **Standardisation des dates** : Harmonisation de 4 formats de timestamps differents (ISO, ISO-T, francais, americain) vers un format datetime uniforme.

3. **Conversion numerique** : Correction des separateurs decimaux (virgule vers point) et remplacement des valeurs textuelles invalides (`""`, `"NA"`, `"null"`) par `NaN`, puis conversion en `float64`.

4. **Correction des valeurs aberrantes** :
   - Temperatures impossibles (hors [-40, 50] C) remplacees par `NaN` pour interpolation ulterieure
   - Humidite clippee a l'intervalle [0, 100]%
   - Rayonnement solaire negatif remis a 0
   - Vitesse de vent et precipitation negatives remises a 0

5. **Traitement des valeurs manquantes** :
   - Interpolation lineaire par commune pour temperature, humidite, rayonnement et vent
   - Forward fill par commune pour les precipitations
   - Suppression des lignes residuelles avec `NaN`

6. **Enrichissement temporel** : Ajout des colonnes `jour`, `mois`, `heure`, `saison` et `jour_semaine` pour faciliter les analyses futures.

7. **Sauvegarde** : Export du DataFrame nettoye vers `../output/meteo_clean.csv`.

Le fichier nettoye est maintenant pret pour les etapes d'analyse et de modelisation (Partie 2.2 et suivantes).